In [23]:
import pandas as pd
import requests
import time
from dateutil.parser import parse

In [24]:
# API endpoint and key for Alpha Vantage
endpoint = "https://www.alphavantage.co/query"
apikey = "RGUX1KUUCDUCRYAC"

In [25]:
# Define time intervals for closing prices
time_intervals = {"1_hour": 60, "1_day": 24*60, "1_week": 7*24*60, "1_mon": 30*24*60}

# Load data frame
data = pd.read_csv("/Users/teekyboy/Desktop/random_test.csv")

In [26]:
# Loop through stock symbols
for symbol in data["stock"]:
    # Get time series data from Alpha Vantage
    params = {"function": "TIME_SERIES_INTRADAY_EXTENDED", "symbol": symbol, "interval": "1min", "apikey": apikey}
    response = requests.get(endpoint, params=params)
    if response.ok:
        data_json = response.json()
        time_series = data_json
        if time_series:
            # Get buying price at the time of the date column
            date_str = data.loc[data["stock"]==symbol, "date"].values[0]
            date = parse(date_str)
            timestamp = int(date.timestamp())
            price = float(slice(time_series, str(timestamp), 0, 5)[0][4])
            if price == "NaN":
                # If price is not available, get opening price of next day
                next_date = date + pd.Timedelta(days=1)
                next_timestamp = int(next_date.timestamp())
                next_day_series = slice(time_series, str(next_timestamp), 0, 5)
                if next_day_series:
                    price = float(next_day_series[0][1])
            # Update price column
            data.loc[data["stock"]==symbol, "price"] = price
            # Get closing prices for different time intervals
            for interval, minutes in time_intervals.items():
                closing_date = date + pd.Timedelta(minutes=minutes)
                closing_timestamp = int(closing_date.timestamp())
                closing_series = slice(time_series, str(closing_timestamp), 0, 5)
                if closing_series:
                    closing_price = float(closing_series[0][4])
                    data.loc[data["stock"]==symbol, f"closing_{interval}"] = closing_price
        else:
            print(f"No time series found for symbol {symbol}")
    else:
        print(f"Error getting data for symbol {symbol}")
    # Sleep for 12 seconds to avoid API rate limit
    time.sleep(12)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [22]:
data.head()

,url,date,tweet,username,likeCount,replyCount,retweetCount,quoteCount,cashtags,hashtags,viewCount,stock,price_at_tweet,price_1_min,1_hour,1_day,1_week,1_month,stocks_without_cashtag,price
0,https://twitter.com/StockMKTNewz/status/147708...,2022-01-01 01:15:57+00:00,I‚Äôm in one of the Tesla‚Äôs $TSLA Revel boug...,StockMKTNewz,42,1,2,0,['TSLA'],NaN,NaN,TSLA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://twitter.com/LilMoonLambo/status/147713...,2022-01-01 04:22:22+00:00,$ETH bears died a painful death this year Let‚...,LilMoonLambo,33,5,4,0,['ETH'],NaN,NaN,ETH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://twitter.com/mikealfred/status/14771427...,2022-01-01 05:00:59+00:00,You can buy BTC with those? üòÇ,mikealfred,3,1,0,0,[],['Bitcoin'],NaN,BTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://twitter.com/Fxhedgers/status/147714392...,2022-01-01 05:05:30+00:00,NEW YORK (AP) ‚Äî 2022 dawns on the U.S. East ...,Fxhedgers,274,1,26,5,[],NaN,NaN,SQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
